<a href="https://colab.research.google.com/github/bhanupra7ap/f-o_model/blob/main/price_predection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

In [40]:
call_data = pd.read_csv('call_data.csv')
put_data = pd.read_csv('put_data.csv')

In [41]:
columns_to_drop = ['Settle Price', 'Symbol', 'Underlying Value', 'Turnover * in  ₹ Lakhs', 'Premium Turnover ** in   ₹ Lakhs']
call_data = call_data.drop(columns=columns_to_drop)
put_data = put_data.drop(columns=columns_to_drop)

data = pd.concat([call_data, put_data])

In [42]:
data['Date'] = pd.to_datetime(data['Date'])
data['Expiry'] = pd.to_datetime(data['Expiry'])

In [43]:
label_encoder = LabelEncoder()
data['Option Type'] = label_encoder.fit_transform(data['Option Type'])

In [44]:
data.replace("-", 0, inplace=True)

data['Open'] = pd.to_numeric(data['Open'], errors='coerce')
data['High'] = pd.to_numeric(data['High'], errors='coerce')
data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['LTP'] = pd.to_numeric(data['LTP'], errors='coerce')
data['No. of contracts'] = pd.to_numeric(data['No. of contracts'], errors='coerce')
data['Open Int'] = pd.to_numeric(data['Open Int'], errors='coerce')
data['Change in OI'] = pd.to_numeric(data['Change in OI'], errors='coerce')
data['Strike Price'] = pd.to_numeric(data['Strike Price'], errors='coerce')

In [45]:
scaler = MinMaxScaler()
data[['Open', 'High', 'Low', 'Close', 'LTP', 'No. of contracts', 'Open Int', 'Change in OI', 'Strike Price']] = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close', 'LTP', 'No. of contracts', 'Open Int', 'Change in OI', 'Strike Price']])

In [46]:
data['LTP_next'] = data['LTP'].shift(-1)
data['Upward_move'] = np.where(data['LTP_next'] > data['LTP'], 1, 0)

In [47]:
data = data.dropna()

In [48]:
X = data[['Open', 'High', 'Low', 'Close', 'No. of contracts', 'Open Int', 'Change in OI', 'Strike Price', 'Option Type']]
y = data['Upward_move']

In [49]:
X = np.array(X)
y = np.array(y)
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [51]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [52]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [53]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.7179 - loss: 0.5704 - val_accuracy: 0.6959 - val_loss: 0.6388
Epoch 2/10
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.7513 - loss: 0.5438 - val_accuracy: 0.6984 - val_loss: 0.6243
Epoch 3/10
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.7579 - loss: 0.5359 - val_accuracy: 0.6932 - val_loss: 0.6192
Epoch 4/10
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7590 - loss: 0.5251 - val_accuracy: 0.6938 - val_loss: 0.6057
Epoch 5/10
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.7591 - loss: 0.5160 - val_accuracy: 0.6977 - val_loss: 0.5893
Epoch 6/10
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.7602 - loss: 0.5100 - val_accuracy: 0.7056 - val_loss: 0.5970
Epoch 7/10
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.7590 - loss: 0.5110 - val_accuracy: 0.7004 - val_loss: 0.5834
Epoch 8/10
2611/2611 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.7597 - loss: 0

In [55]:
loss, accuracy = model.evaluate(X_test, y_test)

653/653 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7093 - loss: 0.5783


In [56]:
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.7083
